# Sigma scraping project

In [1]:
import requests
from bs4 import BeautifulSoup

In [ ]:
sigma_request = requests.get("https://www.sigma-computer.com/search?route=product%2Fsearch&search=laptop&submit_search=")
sigma_request

In [3]:
sigma_soup = BeautifulSoup(sigma_request.text , 'html.parser')
products_list = sigma_soup.find_all(name = 'div' , attrs = {"class" : 'product-layout col-lg-15 col-md-4 col-sm-6 col-xs-12'})

In [ ]:
with open("sigma_laptops.csv", mode='w', encoding='utf-8') as fd:
  fd.write("Title,Product link,Price,Category,Brand,Model,Seller,Viewes,Availability,Processor,GPU,Display,Screen Size,Resolution,Storage,RAM,Cache,Cores/Threads,Battery,Operating System\n")
  for j in range(len(products_list)):
    try:
      print(j)
      product_link = "https://www.sigma-computer.com/" + products_list[j].find(name = 'div' , attrs = {'class' : "caption hide-cont"}).find(name = 'a').attrs['href']
      product_link_response = requests.get(product_link)
      product_link_soup = BeautifulSoup(product_link_response.text , 'html.parser')
      product_categories = product_link_soup.find(name = 'div' , attrs = {'class' : 'tab-content col-lg-12 col-sm-12 col-xs-12'}).find(name = 'div' , attrs = {'id' : 'tab-4'}).find_all(name = 'tr')[1].find_all(name = 'td')[-1].text.strip()
      if(product_categories == 'NOTEBOOK ( LAPTOPS )'):
        product_category = 'Laptop'
        product_price = product_link_soup.find(name = 'span' , attrs = {'class' : 'price-new'}).text.replace(",",".")
        laptop_title = product_link_soup.find(name = 'div' , attrs = {'class' : 'title-product'}).text.replace("\n", '').strip().replace(",",".")
        stock = product_link_soup.find(name = 'button' , attrs = {'id' : 'button-cart'}).text.replace('\n','').strip()
        if stock == 'Add to Cart':
          availability = 'In stock'
        else:
          availability = 'Out of stock'
          
        product_brand = product_link_soup.find(name = 'div' , attrs = {'class' : 'brand'}).find(name = 'a').text.strip().replace(",",".")
        product_model = product_link_soup.find_all(name = 'div' , attrs = {'class' : 'model'})[0].find(name= 'a').text.replace(",",".")
        product_seller = product_link_soup.find_all(name = 'div' , attrs = {'class' : 'model'})[1].find(name= 'a').text.strip().replace(",",".")
        product_viewes = product_link_soup.find_all(name = 'div' , attrs = {'class' : 'model'})[2].text.split("  ")[-1].strip().replace(",",".")

        specifications = product_link_soup.find_all(name = 'h3' , attrs = {'class' : 'modtitle'})[4].find_all(name ='li')
        specifications2 = product_link_soup.find_all(name = 'h3' , attrs = {'class' : 'modtitle'})[4].find_all(name ='p')
        specifications3 = product_link_soup.find_all(name = 'h3' , attrs = {'class' : 'modtitle'})[4].find_all(name ='h6')
        discription = ''

        specs = ['Processor','GPU','Storage','Display','OS']

        if (len(specifications) != 0):
          specifications_dic = {}
          for i in range(len(specifications)):
            specs_key = specifications[i].find_all(name = 'strong')
            if(len(specs_key) != 0):
              specs_key = specifications[i].find_all(name = 'strong')[0].text.replace(":","").strip()
            else:
              if(i >= len(specs)):
                break
              else:
                specifications_dic[specs[i]] = specifications[i+1].text.replace('\n','').strip().replace('\t',"").replace(":","")
                continue
            specifications_dic[specs_key] = specifications[i].text.replace("\n" , "").replace('\t',"")
            indexx = specifications_dic[specs_key].find('\r')
            if (indexx != -1):
              specifications_dic[specs_key] = (specifications_dic[specs_key])[0:indexx]
            specifications_dic[specs_key] = specifications_dic[specs_key].split(":")[-1].strip()
            
        elif (len(specifications2) != 0):
          specifications_dic = {}
          for i in range(len(specifications2)):
            specs_key = specifications2[i].find_all(name = 'strong')
            if(len(specs_key) != 0):
              specs_key = specifications2[i].find_all(name = 'strong')[0].text.replace(":","").strip()
            else:
              if(i >= len(specs)):
                break
              else:
                specifications_dic[specs[i]] = specifications2[i].text.replace('\n','').strip().replace('\t',"").replace(":","")
                continue
            specifications_dic[specs_key] = specifications2[i].text.replace("\n" , "").replace('\t',"")
            indexx = specifications_dic[specs_key].find('\r')
            if (indexx != -1):
              specifications_dic[specs_key] = (specifications_dic[specs_key])[0:indexx]
            specifications_dic[specs_key] = specifications_dic[specs_key].split(":")[-1].strip()

        elif (len(specifications3) != 0):
          specifications_dic = {}
          for i in range(len(specifications3)):
            specs_key = specifications3[i].find_all(name = 'strong')
            if(len(specs_key) != 0):
              specs_key = specifications3[i].find_all(name = 'strong')[0].text.replace(":","").strip()
            else:
              if(i >= len(specs)):
                break
              else:
                specifications_dic[specs[i]] = specifications[i].text.replace('\n','').strip().replace('\t',"").replace(":","")
                continue
            specifications_dic[specs_key] = specifications3[i].text.replace("\n" , "").replace('\t',"")
            indexx = specifications_dic[specs_key].find('\r')
            if (indexx != -1):
              specifications_dic[specs_key] = (specifications_dic[specs_key])[0:indexx]
            specifications_dic[specs_key] = specifications_dic[specs_key].split(":")[-1].strip()

        else:
          specifications4 = product_link_soup.find_all(name = 'h3' , attrs = {'class' : 'modtitle'})[4].find_all(name ='div')
          if(" " in specifications4):
            specifications4.remove(" ")
          if("" in specifications4):
            specifications4.remove("")
          specifications_dic = {}
          c = 0;
          for i in range(1,len(specifications4)-2,3):
            if(c >= len(specs)):
              break
            else:
              specifications_dic[specs[c]] = specifications4[i].text.replace('\n','').strip().replace('\t',"").replace(":","").replace("ضمـــــــــان سنــــة","") + specifications4[i+1].text.replace('\n','').strip().replace('\t',"").replace(":","") + specifications4[i+2].text.replace('\n','').strip().replace('\t',"").replace(":","") 
              c += 1

        if 'Processor' in specifications_dic.keys():
          laptop_processor = specifications_dic['Processor'].replace(",",".")
        elif len(list(specifications_dic.keys())) == 0 and len(discription) != 0:
          laptop_processor = discription
        elif "Processor (CPU)" in specifications_dic.keys():
          laptop_processor = specifications_dic['Processor (CPU)'].replace(",",".")
        else:
          laptop_processor = 'Not mentioned'

        if 'GPU' in specifications_dic.keys():
          laptop_GPU = specifications_dic['GPU'].replace(",",".")
        elif 'Graphics Card' in specifications_dic.keys():
          laptop_GPU = specifications_dic['Graphics Card'].replace(",",".")
        else:
          laptop_GPU = 'Not mentioned'

        if 'Display' in specifications_dic.keys():
          laptop_Display = specifications_dic['Display'].replace(",",".")
        else:
          laptop_Display = 'Not mentioned'

        if 'Screen' in specifications_dic.keys():
          laptop_screen = specifications_dic['Screen'].replace(",",".")
        elif 'Screen Size' in specifications_dic.keys():
          laptop_screen = specifications_dic['Screen Size'].replace(",",".")
        else:
          laptop_screen = 'Not mentioned'

        if 'Resolution' in specifications_dic.keys():
          laptop_Resolution = specifications_dic['Resolution'].replace(",",".")
        else:
          laptop_Resolution = 'Not mentioned'

        if 'Memory' in specifications_dic.keys():
          laptop_Memory = specifications_dic['Memory'].replace(",",".")
        elif 'Storage' in specifications_dic.keys():
          laptop_Memory = specifications_dic['Storage'].replace(",",".")
        else:
          laptop_Memory = 'Not mentioned'

        if 'RAM' in specifications_dic.keys():
          laptop_RAM = specifications_dic['RAM'].replace(",",".")
        else:
          laptop_RAM = 'Not mentioned'

        if 'Cores/Threads' in specifications_dic.keys():
          laptop_cores = specifications_dic['Cores/Threads'].replace(",",".")
        else:
          laptop_cores = 'Not mentioned'

        if 'Cache' in specifications_dic.keys():
          laptop_Cache = specifications_dic['Cache'].replace(",",".")
        else:
          laptop_Cache = 'Not mentioned'

        if 'Battery' in specifications_dic.keys():
          laptop_Battery = specifications_dic['Battery'].replace(",",".")
        else:
          laptop_Battery = 'Not mentioned'

        if 'Operating System' in specifications_dic.keys():
          laptop_OS = specifications_dic['Operating System'].replace(",",".")
        else:
          laptop_OS = 'Not mentioned'
        
        fd.write(f"\"{laptop_title}\",\"{product_link}\",\"{product_price}\",\"{product_category}\",\"{product_brand}\",\"{product_model}\",\"{product_seller}\",\"{product_viewes}\",\"{availability}\",\"{laptop_processor}\",\"{laptop_GPU}\",\"{laptop_Display}\",\"{laptop_screen}\",\"{laptop_Resolution}\",\"{laptop_Memory}\",\"{laptop_RAM}\",\"{laptop_Cache}\",\"{laptop_cores}\",\"{laptop_Battery}\",\"{laptop_OS}\"\n")
    except:
      print("Error happend")


In [1]:
import pandas as pd
df  = pd.read_csv("sigma_laptops.csv")
df

,Title,Product link,Price,Category,Brand,Model,Seller,Viewes,Availability,Processor,GPU,Display,Screen Size,Resolution,Storage,RAM,Cache,Cores/Threads,Battery,Operating System
0,HP Victus16-r1062ne Intel Core i5 14450HX-RTX ...,https://www.sigma-computer.com/item?id=7642&na...,43999 LE,Laptop,HP,r1062ne,sigma,1475 items,In stock,Intel Core i5-14450HX,NVIDIA GeForce RTX 3050 with 6GB GDDR6 VRAM,16.1-inch FHD IPS display with a 144Hz refresh...,Not mentioned,Not mentioned,1TB SSD Gen4 (ultra-fast NVMe storage for quic...,16GB DDR5 (2 x 8GB modules) with a blazing-fas...,24MB,10 cores (6 performance cores + 4 efficient co...,Not mentioned,Not mentioned
1,HP Victus16-r1066ne Intel Core i5 14450HX-RTX ...,https://www.sigma-computer.com/item?id=7641&na...,43499 LE,Laptop,HP,r1066ne,sigma,1217 items,In stock,Intel Core i5-14450HX,NVIDIA GeForce RTX 3050 with 6GB of VRAM,Not mentioned,Not mentioned,Not mentioned,Graphics Card NVIDIA GeForce RTX 3050 with 6GB...,16GB DDR5 (2 x 8GB modules) at 5600MHz,Not mentioned,10 cores (6 performance + 4 efficiency) / 16 t...,Not mentioned,Not mentioned
2,MSI Modern 15 F13MG Laptop - Intel Core i5-133...,https://www.sigma-computer.com/item?id=7624&na...,21749 LE,Laptop,MSI,F13MG,sigma,1835 items,In stock,Intel Core i5-1335U (13th Gen. up to 4.60 GHz)...,Not mentioned,"15.6 FHD (1920x1080)""",Not mentioned,Not mentioned,512GB NVMe SSD,8GB DDR4,Not mentioned,Not mentioned,Not mentioned,Not mentioned
3,Lenovo LOQ 15IRX9 83DV0071PS Laptop Intel Cor...,https://www.sigma-computer.com/item?id=7600&na...,46499 LE,Laptop,Lenovo,83DV0071PS,sigma,2891 items,Out of stock,Intel® Core™ i7-13650HX (14C/20T. up to 4.9GHz),NVIDIA® GeForce RTX™ 4050 (6GB GDDR6. 105W TGP...,"15.6 FHD IPS. 144Hz. 100% sRGB""",Not mentioned,Not mentioned,16GB DDR5-4800 (upgradable to 32GB),Not mentioned,Not mentioned,Not mentioned,60Wh,Not mentioned
4,MSI-I9-Vector16 HX A14VHG-801EG Inte Core i9-1...,https://www.sigma-computer.com/item?id=7583&na...,133999 LE,Laptop,MSI,Vector 16 HX A14V,sigma,1394 items,In stock,Intel Core i9-14900HX (14 cores. 24 threads),NVIDIA GeForce RTX 4080 (12GB GDDR6),16-inch QHD+ (2560x1600). 240Hz refresh rate. ...,Not mentioned,Not mentioned,1TB NVMe PCIe Gen4x4 SSD,32GB DDR5 (expandable),Not mentioned,Not mentioned,Not mentioned,Windows 11 Home
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
529,HP ProBook 450 G6 Laptop - Intel Core I7 - 8GB...,https://www.sigma-computer.com/item?id=452&nam...,12600 LE,Laptop,HP,HP ProBook 450 G6,sigma,4871 items,Out of stock,8GB DDR4 RAM‎.‎ 1TB HDD,15.6‎-inch HD Anti‎-Glare LED‎-backlit Screen,Wi‎-Fi 802.11a/b/g/n/ac‎.‎ Bluetooth 5.0,Not mentioned,Not mentioned,2GB NVIDIA GeForce MX130 GPU,Not mentioned,Not mentioned,Not mentioned,Not mentioned,Not mentioned
530,HP 15-da0122ne Laptop - Intel Core I3 - 4GB RA...,https://www.sigma-computer.com/item?id=451&nam...,5800 LE,Laptop,HP,HP 15-da0122ne,sigma,5177 items,Out of stock,4GB DDR4 RAM‎.‎ 1TB HDD,15.6‎-inch HD WLED‎-backlit Screen,Wi‎-Fi 802.11b/g/n/ac‎.‎ Bluetooth 4.2,Not mentioned,Not mentioned,2 GB NVIDIA GeForce MX110 GPU,Not mentioned,Not mentioned,Not mentioned,Not mentioned,Not mentioned
531,Inspiron 15-7577 Gaming Laptop - Intel Core i7...,https://www.sigma-computer.com/item?id=321&nam...,18500 LE,Laptop,Dell,NaN,sigma,8813 items,Out of stock,16GB RAM. 512GB SSD + 1TB HDD,"15.6 Full HD LED-Backlit Screen""",Wi-Fi 802.11ac. Bluetooth 4.2,Not mentioned,Not mentioned,6GB NVIDIA GeForce GTX 1060 GPU,Not mentioned,Not mentioned,Not mentioned,Not mentioned,Not mentioned
532,G3 15-3579 Gaming Laptop - Intel Core I7 - 8GB...,https://www.sigma-computer.com/item?id=271&nam...,15000 LE,Laptop,Dell,3579,sigma,4835 items,Out of stock,8GB RAM.128GB SSD + 1TB HDD,15.6-inch FHD IPS LED-Backlit Screen,Wi-Fi 802.11ac. Bluetooth 5.0,Not mentioned,Not mentioned,4GB NVIDIA GeForce GTX 1050 Ti GPU,Not mentioned,Not mentioned,Not mentioned,Not mentioned,Not mentioned
